In [1]:
import torch
from modules.modules import HyperNet, MainNet
import ikpy.chain
import json
import numpy as np
import os


In [2]:
# Runtime Configuration defaults
class Config:
    chain_path='assets/UR5/urdf/ur5_robot.urdf'
    train_data_path='data/ur5/ur5_train_data.csv'
    test_data_path='data/ur5/ur5_test_data.csv'
    num_joints=6
    lr=0.001
    num_epochs=200
    num_solutions_validation=10
    batch_size=2048
    early_stopping_epochs=50
    grad_clip=1
    embedding_dim=128
    hypernet_input_dim=6
    hypernet_hidden_size=1024
    hypernet_num_hidden_layers=3
    jointnet_hidden_size=256
    num_gaussians=50
    exp_dir="runs"
    jointnet_output_dim=150
    jointnet_output_dim = 2 if num_gaussians == 1 else num_gaussians * 2 + num_gaussians

cfg = Config()


In [3]:
def load_json_config(json_path):

    with open(json_path, 'r') as f:
        config_dict = json.load(f)
    return config_dict

In [4]:
def update_config(config_dict, config_class):

    for key, value in config_dict.items():
        if hasattr(config_class, key):
            setattr(config_class, key, value)
        else:
            print(f"Warning: '{key}' not found in the class. Skipping.")

In [9]:
run_args_path = "runs/exp_10/run_args.json"

In [10]:
run_args = load_json_config(run_args_path)

In [11]:
update_config(run_args, cfg)

In [12]:
# Initialize HyperNet and MainNet
hypernet = HyperNet(cfg)
mainnet = MainNet(cfg)


In [13]:
# Load the best model weights into HyperNet
model_path = "runs/exp_10/best_model.pt"
hypernet.load_state_dict(torch.load(model_path))
hypernet.eval()  # Set the model to evaluation mode

C:\Users\SAE\AppData\Local\Temp\ipykernel_19372\1268630429.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hypernet.load_state_dict(torch.load(model_path))


HyperNet(
  (layers): ModuleList(
    (0): Linear(in_features=6, out_features=1024, bias=True)
    (1-2): 2 x Linear(in_features=1024, out_features=1024, bias=True)
  )
  (out): Linear(in_features=1024, out_features=128, bias=True)
  (projection): MultiHeadLinearProjection(
    (linears): ModuleList(
      (0-1): 2 x ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=256, bias=True)
        )
      )
      (2): ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=38400, bias=True)
        )
      )
      (3): ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=150, bias=True)
        )
      )
      (4): ProjectionHead(
        (head): Sequential(
          (linear_final): Linear(in_features=128, out_features=512, bias=True)
        )
      )
      (5): ProjectionHead(
        (head): Sequential(
          (linear_final): L

In [14]:
# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hypernet = hypernet.to(device)
mainnet = mainnet.to(device)

In [15]:
# Load the kinematic chain for FK calculations (optional)
r_arm = ikpy.chain.Chain.from_urdf_file(cfg.chain_path)

# Extract joint limits
upper = []
lower = []
for i in range(1, len(r_arm.links) - 1):
    lower.append(r_arm.links[i].bounds[0])
    upper.append(r_arm.links[i].bounds[1])

upper = np.array(upper)
lower = np.array(lower)


c:\Users\SAE\Desktop\Organisatorisches\FH\AI\3. Semester\ROBOTICS\robotics_project\.venv\Lib\site-packages\ikpy\chain.py:60: UserWarning: Link Base link (index: 0) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive".format(link.name, link_index))
c:\Users\SAE\Desktop\Organisatorisches\FH\AI\3. Semester\ROBOTICS\robotics_project\.venv\Lib\site-packages\ikpy\chain.py:60: UserWarning: Link ee_fixed_joint (index: 7) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't provide any transformation so is as it were inactive
  warnings.warn("Link {} (index: {}) is of type 'fixed' but set as active in the active_links_mask. In practice, this fixed link doesn't pr

In [17]:
# Example input position (e.g., desired end-effector position)

positions = torch.tensor([[0.009839, -0.123490, 0.046488, 0.448670, 0.547631, -0.706255]], dtype=torch.float32)  # first 3 are positions (TCP), last 3 are orientations 
positions = positions.to(device)


In [18]:
# Predict weights using HyperNet
with torch.no_grad():  # Disable gradient computation
    predicted_weights = hypernet(positions)

# Generate joint angles using MainNet
with torch.no_grad():
    initial_input = torch.ones((positions.shape[0], 1), dtype=torch.float32).to(device)
    samples, distributions, means, variance, selection = mainnet.validate(
        initial_input, predicted_weights, lower, upper
    )

# Convert the predicted joint angles to a readable format
predicted_joint_angles = []
for sample in samples:
    predicted_joint_angles.append([angle.item() for angle in sample])


In [10]:
predicted_joint_angles

[[-2.54919695854187],
 [0.29687026143074036],
 [2.5],
 [0.31112515926361084],
 [-2.040735960006714],
 [1.6099227666854858]]

In [19]:
# Flatten the predicted joint angles
flat_joint_angles = [angle[0] for angle in predicted_joint_angles]


In [20]:
# Add base and end-effector placeholders (if necessary)
full_joint_angles = [0] + flat_joint_angles + [0]  # Base and end-effector placeholders

In [21]:
full_joint_angles

[0,
 3.032597541809082,
 0.6473861932754517,
 -2.5,
 -3.1415927410125732,
 -0.46253931522369385,
 0.728458046913147,
 0]

In [22]:
len(r_arm.links) 

8

In [23]:


# Check for length mismatch
if len(full_joint_angles) != len(r_arm.links):
    raise ValueError("Mismatch between joint angles and kinematic chain.")

# Compute Forward Kinematics
fk_position = r_arm.forward_kinematics(full_joint_angles)[:3, 3]  # Extract end-effector position

In [24]:

print(f"FK Position using predicted joints: {fk_position}")
print(f"original position: {positions[0]}")


FK Position using predicted joints: [-0.14791288 -0.16770742  0.21856937]
original position: tensor([ 0.0098, -0.1235,  0.0465,  0.4487,  0.5476, -0.7063], device='cuda:0')


In [27]:
def count_model_parameters(model):

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    return {
        "total_params": total_params,
        "trainable_params": trainable_params,
        "non_trainable_params": total_params - trainable_params
    }

In [26]:
params = count_model_parameters(hypernet)
params

{'total_params': 32966916,
 'trainable_params': 32966916,
 'non_trainable_params': 0}

In [4]:
from datasets.dataset_csv import IKDatasetValCSV
dataloader = IKDatasetValCSV("ur5_val_data_87k copy.csv")

In [10]:
def calculate_accuracy(hypernet, mainnet, dataloader, r_arm, lower, upper, threshold_position=0.1):

    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    hypernet.eval()
    mainnet.eval()

    with torch.no_grad():
        for positions, joint_angles in dataloader:
            positions = positions.to(device).view(-1, positions.shape[-1])
            joint_angles = joint_angles.to(device).view(-1, joint_angles.shape[-1])

            # Split positions into positional (x, y, z) and orientation (rx, ry, rz) components
            pos_xyz = positions[:, :3]  # x, y, z
            ori_xyz = positions[:, 3:]  # rx, ry, rz

            # Predict mixture weights with hypernet using the full position + orientation input
            predicted_weights = hypernet(positions)

            # Generate joint angle predictions using mainnet
            initial_input = torch.ones((positions.shape[0], 1), dtype=torch.float32).to(device)
            samples, _, _, _, _ = mainnet.validate(initial_input, predicted_weights, lower, upper)

            for i in range(len(pos_xyz)):
                # Construct joint angles for forward kinematics
                flat_joint_angles = [sample[i].item() for sample in samples]
                full_joint_angles = [0] + flat_joint_angles + [0]  # Add base and end-effector placeholders

                # Compute FK for position
                fk_matrix = r_arm.forward_kinematics(full_joint_angles)
                fk_position = fk_matrix[:3, 3]

                # Compare with target end-effector position
                target_position = pos_xyz[i].cpu().numpy()
                error_position = np.linalg.norm(fk_position - target_position)

                # Count as correct if position is within threshold
                if error_position < threshold_position:
                    correct += 1
                total += 1

    accuracy = correct / total * 100 if total > 0 else 0
    print(f"Accuracy: {accuracy:.2f}% (position threshold: {threshold_position} meters)")
    return accuracy


In [11]:
accuracy = calculate_accuracy(hypernet, mainnet, dataloader, r_arm, lower, upper) # IK8

Accuracy: 52.25% (position threshold: 0.1 meters)
